# XGBoost Model

In [1]:
import pandas as pd
from xgboost import XGBRegressor, callback
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import optuna

In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v7.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v7.csv")

# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']

n_jobs = os.cpu_count() // 2

In [ ]:
# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the objective function to be used by Optuna for hyperparameter optimization
def objective(trial):
    # Suggest values for the hyperparameters to be optimized
    params = {
        'objective': 'reg:squarederror',
        'random_state': 42,
        'n_estimators': 1_000_000,  # Set a high value to allow early stopping
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-3, 1.0, log=True),
        'subsample': 1.0,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 1.0, log=True),
        'n_jobs': n_jobs,
        'eval_metric': 'rmse',  # Move eval_metric to the constructor
        'early_stopping_rounds': 20  # Move early_stopping_rounds to the constructor
    }
    
    # Initialize the model with the suggested parameters
    model = XGBRegressor(**params)
    
    # Train the model with early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=1000
    )
    
    # Make predictions and calculate the error on the validation set
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    
    return rmse

# Start the study with Optuna
study = optuna.create_study(direction='minimize')  # Minimize the RMSE
study.optimize(objective, n_trials=50)  # Adjust the number of trials as necessary

# Display the best hyperparameters found
print(f"Best trial: {study.best_trial.params}")

# Train the final model with the best parameters
best_params = study.best_trial.params
best_model = XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    random_state=42,
    n_estimators=1_000_000,
    n_jobs=n_jobs
)

# Train the model with early stopping
best_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=500
)

# Update the n_estimators parameter with the best value found
best_params['n_estimators'] = best_model.best_iteration + 1

# Evaluate the performance on the test set
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")
print(f"Updated best_params: {best_params}")


[I 2024-09-13 03:14:17,805] A new study created in memory with name: no-name-5e934a07-50a3-4200-96ca-372cbbbfbcac


[0]	validation_0-rmse:53028.08490
[1000]	validation_0-rmse:3403.39350
[2000]	validation_0-rmse:3337.55713
[3000]	validation_0-rmse:3312.21298
[4000]	validation_0-rmse:3300.42199
[4605]	validation_0-rmse:3296.37625


[I 2024-09-13 03:30:41,925] Trial 0 finished with value: 3296.3675170364886 and parameters: {'learning_rate': 0.007569165081214165, 'max_depth': 15, 'min_child_weight': 8, 'gamma': 0.10997095295835893, 'colsample_bytree': 0.5415703588078289, 'reg_alpha': 0.7476742564840364, 'reg_lambda': 0.00010521677052091404}. Best is trial 0 with value: 3296.3675170364886.


[0]	validation_0-rmse:52880.97856
[1000]	validation_0-rmse:3377.56768
[2000]	validation_0-rmse:3331.23844
[3000]	validation_0-rmse:3312.38523
[4000]	validation_0-rmse:3303.20366
[5000]	validation_0-rmse:3297.99779
[5973]	validation_0-rmse:3294.88116


[I 2024-09-13 03:53:18,881] Trial 1 finished with value: 3294.869499072892 and parameters: {'learning_rate': 0.010364167332776215, 'max_depth': 15, 'min_child_weight': 1, 'gamma': 0.06250594645395556, 'colsample_bytree': 0.4340437646679536, 'reg_alpha': 0.255983719187635, 'reg_lambda': 0.3780729793233056}. Best is trial 1 with value: 3294.869499072892.


...

[0]	validation_0-rmse:53364.96537
[1000]	validation_0-rmse:15895.70869
[2000]	validation_0-rmse:5868.27268
[3000]	validation_0-rmse:3944.57396
[4000]	validation_0-rmse:3677.29341
[5000]	validation_0-rmse:3603.31388
[6000]	validation_0-rmse:3562.12038
[7000]	validation_0-rmse:3530.97463
[8000]	validation_0-rmse:3508.30033
[9000]	validation_0-rmse:3492.53487
[10000]	validation_0-rmse:3478.95158
[11000]	validation_0-rmse:3467.99328
[12000]	validation_0-rmse:3458.45913
[13000]	validation_0-rmse:3448.84037
[14000]	validation_0-rmse:3440.21068
[15000]	validation_0-rmse:3431.51352
[16000]	validation_0-rmse:3423.85437
[17000]	validation_0-rmse:3417.37356
[18000]	validation_0-rmse:3411.73976
[19000]	validation_0-rmse:3406.45118
[20000]	validation_0-rmse:3401.62617
[21000]	validation_0-rmse:3396.70903
[22000]	validation_0-rmse:3391.36122
[23000]	validation_0-rmse:3387.48879
[24000]	validation_0-rmse:3383.25795
[25000]	validation_0-rmse:3378.37539
[26000]	validation_0-rmse:3375.12639
[27000]	vali

[I 2024-09-13 09:53:32,193] Trial 12 finished with value: 3331.148438043154 and parameters: {'learning_rate': 0.0012391954223833696, 'max_depth': 11, 'min_child_weight': 3, 'gamma': 0.012318717038533471, 'colsample_bytree': 0.8020871813838039, 'reg_alpha': 0.0008367835099811545, 'reg_lambda': 0.026581056419018298}. Best is trial 9 with value: 3288.939801874509.


[0]	validation_0-rmse:53281.80250
[1000]	validation_0-rmse:4897.57536
[2000]	validation_0-rmse:3564.81110
[3000]	validation_0-rmse:3475.89165
[4000]	validation_0-rmse:3432.11165
[5000]	validation_0-rmse:3406.42391
[6000]	validation_0-rmse:3386.12259
[7000]	validation_0-rmse:3370.44130
[8000]	validation_0-rmse:3357.46677
[9000]	validation_0-rmse:3347.01733
[10000]	validation_0-rmse:3337.23279
[11000]	validation_0-rmse:3330.02285
[12000]	validation_0-rmse:3324.32212
[13000]	validation_0-rmse:3319.80215
[14000]	validation_0-rmse:3316.38339
[15000]	validation_0-rmse:3312.25495
[16000]	validation_0-rmse:3309.21821
[16158]	validation_0-rmse:3308.89570


[I 2024-09-13 10:26:55,274] Trial 13 finished with value: 3308.874359910568 and parameters: {'learning_rate': 0.0028037697665779446, 'max_depth': 12, 'min_child_weight': 6, 'gamma': 0.0012940687187252568, 'colsample_bytree': 0.6323190991378431, 'reg_alpha': 0.04436462408705523, 'reg_lambda': 0.001427818027213409}. Best is trial 9 with value: 3288.939801874509.


[0]	validation_0-rmse:53284.67403
[1000]	validation_0-rmse:5282.65406
[2000]	validation_0-rmse:3829.45696
[3000]	validation_0-rmse:3701.22490
[4000]	validation_0-rmse:3639.15995
[5000]	validation_0-rmse:3592.49959
[6000]	validation_0-rmse:3557.50944
[7000]	validation_0-rmse:3530.16766


In [ ]:
# Save R², RMSE, and hyperparameters
results = {
    'R2': float(r2),
    'RMSE': float(rmse),
    'Best Parameters': {key: (int(value) if isinstance(value, np.integer) else float(value)
                              if isinstance(value, np.floating) else value)
                        for key, value in best_params.items()}
}

# Set timezone to São Paulo (UTC-3)
saopaulo_tz = pytz.timezone('America/Sao_Paulo')
timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')

# Define logs directory, and create them if they don't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Define file paths within the respective directories
results_file = os.path.join(logs_dir, f'model_results_{timestamp}.txt')

# Save the results to a TXT file
with open(results_file, 'w') as file:
    file.write(f"R2: {results['R2']}\n")
    file.write(f"RMSE: {results['RMSE']}\n")
    file.write("Best Parameters:\n")
    for param, value in results['Best Parameters'].items():
        file.write(f"  {param}: {value}\n")

print(f"Results saved to {results_file}")

In [ ]:
# Display evaluation metrics
print(f"Final Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")

In [ ]:
# Define models directory, and create them if they don't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Train the final model using the full training+validation+test set with the optimal n_estimators
final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42, n_jobs=n_jobs)

# Train the model on the entire training+validation+set data
final_model.fit(X, y, verbose=True)

print("Final model trained successfully using all available data.")

In [ ]:
# Define file paths within the respective directories
model_file = os.path.join(models_dir, f'trained_model_{timestamp}.joblib')

# Save the trained model to a file in the models folder
joblib.dump(final_model, model_file)
print(f"Model saved to {model_file}")

In [ ]:
# Use the final model to predict the `tow` for the submission_set_updated
submission_set_features = submission_set_updated.iloc[:,:-1]
submission_set['tow'] = final_model.predict(submission_set_features)

submission_set

In [ ]:
submission_set['tow'].describe()

In [ ]:
challenge_set_updated['tow'].describe()

In [ ]:
# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)